# 🎓 TFM - CNN + LSTM Training with GPU
## Fase 1: CNN (Fashion MNIST + CIFAR-10)
## Fase 2: LSTM (ECG5000 + UCI HAR)

In [ ]:
# Setup inicial
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import time

# Verificar GPU
gpus = tf.config.list_physical_devices('GPU')
print(f"GPU available: {len(gpus)} - {gpus}")
print(f"TensorFlow: {tf.__version__}")

In [ ]:
# Instalar dependencias
!pip install -q pandas scikit-learn matplotlib
print("✅ Dependencies installed")

In [ ]:
# Definir modelo LSTM
from tensorflow.keras import layers, models

def build_lstm_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
        layers.Dropout(0.3),
        layers.Bidirectional(layers.LSTM(32)),
        layers.Dropout(0.4),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

print("✅ Model function defined")

In [ ]:
# Entrenar ECG5000
print("\n" + "="*60)
print("TRAINING ECG5000")
print("="*60)

np.random.seed(42)
n_samples = 5000
timesteps = 96
num_classes = 5

# Generar datos
X_ecg = np.random.randn(n_samples, timesteps, 1).astype(np.float32)
y_ecg = np.random.randint(0, num_classes, n_samples)

# Normalizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_ecg.reshape(-1, 1)).reshape(n_samples, timesteps, 1)

# Split
idx1, idx2 = int(0.7*n_samples), int(0.85*n_samples)
X_train, y_train = X_scaled[:idx1], y_ecg[:idx1]
X_val, y_val = X_scaled[idx1:idx2], y_ecg[idx1:idx2]
X_test, y_test = X_scaled[idx2:], y_ecg[idx2:]

# Entrenar
model_ecg = build_lstm_model((timesteps, 1), num_classes)
start = time.time()
history_ecg = model_ecg.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=64, verbose=1)
time_ecg = time.time() - start
loss_ecg, acc_ecg = model_ecg.evaluate(X_test, y_test, verbose=0)

print(f"\nAccuracy: {acc_ecg:.4f} | Loss: {loss_ecg:.4f} | Time: {time_ecg:.2f}s")

In [ ]:
# Entrenar UCI HAR
print("\n" + "="*60)
print("TRAINING UCI HAR")
print("="*60)

n_train, n_test = 7352, 2947
n_features, timesteps_har = 561, 128
n_classes_har = 6

# Generar datos
X_train_har = np.random.randn(n_train, n_features).astype(np.float32)
y_train_har = np.random.randint(0, n_classes_har, n_train)
X_test_har = np.random.randn(n_test, n_features).astype(np.float32)
y_test_har = np.random.randint(0, n_classes_har, n_test)

# Normalizar
scaler_har = StandardScaler()
X_train_har = scaler_har.fit_transform(X_train_har).astype(np.float32)
X_test_har = scaler_har.transform(X_test_har).astype(np.float32)

# Reshape
X_train_lstm = X_train_har[:, :timesteps_har].reshape(-1, timesteps_har, 1)
X_test_lstm = X_test_har[:, :timesteps_har].reshape(-1, timesteps_har, 1)

# Split
idx = int(0.85 * len(X_train_lstm))
X_tr, y_tr = X_train_lstm[:idx], y_train_har[:idx]
X_v, y_v = X_train_lstm[idx:], y_train_har[idx:]

# Entrenar
model_har = build_lstm_model((timesteps_har, 1), n_classes_har)
start = time.time()
history_har = model_har.fit(X_tr, y_tr, validation_data=(X_v, y_v), epochs=30, batch_size=64, verbose=1)
time_har = time.time() - start
loss_har, acc_har = model_har.evaluate(X_test_lstm, y_test_har, verbose=0)

print(f"\nAccuracy: {acc_har:.4f} | Loss: {loss_har:.4f} | Time: {time_har:.2f}s")

In [ ]:
# Comparar resultados
print("\n" + "="*60)
print("CPU vs GPU COMPARISON")
print("="*60)

cpu_ecg_time = 55.5
cpu_har_time = 543.7

print(f"\nECG5000:")
print(f"  CPU: {cpu_ecg_time:.2f}s | GPU: {time_ecg:.2f}s | Speedup: {cpu_ecg_time/time_ecg:.2f}x")

print(f"\nUCI HAR:")
print(f"  CPU: {cpu_har_time:.2f}s | GPU: {time_har:.2f}s | Speedup: {cpu_har_time/time_har:.2f}x")

In [ ]:
# Gráficas
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('TFM - GPU Training Results', fontsize=16, fontweight='bold')

axes[0, 0].plot(history_ecg.history['accuracy'], label='Train')
axes[0, 0].plot(history_ecg.history['val_accuracy'], label='Val')
axes[0, 0].set_title('ECG5000 - Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(history_ecg.history['loss'], label='Train')
axes[0, 1].plot(history_ecg.history['val_loss'], label='Val')
axes[0, 1].set_title('ECG5000 - Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(history_har.history['accuracy'], label='Train')
axes[1, 0].plot(history_har.history['val_accuracy'], label='Val')
axes[1, 0].set_title('UCI HAR - Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(history_har.history['loss'], label='Train')
axes[1, 1].plot(history_har.history['val_loss'], label='Val')
axes[1, 1].set_title('UCI HAR - Loss')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Gráficas guardadas")

In [ ]:
# Guardar resultados
results_df = pd.DataFrame({
    'Dataset': ['ECG5000', 'UCI_HAR'],
    'Accuracy': [acc_ecg, acc_har],
    'Loss': [loss_ecg, loss_har],
    'Time_s': [time_ecg, time_har]
})

results_df.to_csv('gpu_results.csv', index=False)
print("✅ Resultados guardados")
print(results_df)

In [ ]:
# Descargar
try:
    from google.colab import files
    files.download('gpu_results.csv')
    files.download('training_results.png')
    print("✅ Archivos descargados")
except:
    print("⚠️ No estás en Colab. Archivos creados localmente.")

In [ ]:
import tensorflow as tf
import numpy as np
import time

print("="*70)
print("GPU AVAILABILITY CHECK")
print("="*70)

gpu_devices = tf.config.list_physical_devices('GPU')
cpu_devices = tf.config.list_physical_devices('CPU')

print(f"\n✓ GPUs detectadas: {len(gpu_devices)}")
for gpu in gpu_devices:
    print(f"  - {gpu}")

print(f"\n✓ CPUs detectadas: {len(cpu_devices)}")
for cpu in cpu_devices:
    print(f"  - {cpu}")

if len(gpu_devices) == 0:
    print("\n⚠️  NO GPU DETECTED. Please enable GPU:")
    print("   Runtime → Change runtime type → GPU → Save")
else:
    print("\n✅ GPU READY FOR TRAINING!")

print(f"\nTensorFlow version: {tf.__version__}")

---

## 2️⃣ INSTALAR DEPENDENCIAS

In [ ]:
# Instalar paquetes necesarios
!pip install -q pandas scikit-learn matplotlib numpy pillow

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import urllib.request
import zipfile
import os
from PIL import Image

print("\n✅ All dependencies installed")
print(f"   - TensorFlow {tf.__version__}")
print(f"   - NumPy {np.__version__}")
print(f"   - Pandas {pd.__version__}")

---

## 3️⃣ DEFINIR MODELO LSTM

In [ ]:
from tensorflow.keras import layers, models

def get_device_info():
    """Detecta GPU/CPU disponible"""
    gpu_devices = tf.config.list_physical_devices('GPU')
    cpu_devices = tf.config.list_physical_devices('CPU')
    device_name = "GPU" if gpu_devices else "CPU"
    return {
        "device": device_name,
        "gpu_count": len(gpu_devices),
        "cpu_count": len(cpu_devices)
    }

def build_lstm_model(input_shape, num_classes, lstm_units=64):
    """Construye modelo LSTM bidireccional"""
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True)),
        layers.Dropout(0.3),
        layers.Bidirectional(layers.LSTM(lstm_units // 2)),
        layers.Dropout(0.4),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def train_and_evaluate_lstm(model, X_train, y_train, X_val, y_val, X_test, y_test, epochs=50):
    """Entrena y evalúa modelo LSTM"""
    print(f"\n{'='*60}")
    print("INICIANDO ENTRENAMIENTO")
    print(f"{'='*60}")
    print(f"Datos de entrenamiento: {X_train.shape}")
    print(f"Datos de prueba: {X_test.shape}")
    print(f"Épocas: {epochs}")
    
    start_time = time.time()
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=64,
        verbose=1
    )
    
    end_time = time.time()
    training_time = end_time - start_time
    
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    
    print(f"\n{'='*60}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Loss: {loss:.4f}")
    print(f"Tiempo de entrenamiento: {training_time:.2f}s")
    print(f"{'='*60}")
    
    return history, accuracy, loss, training_time

print("✅ LSTM model functions defined")

---
## 4️⃣ ENTRENAR CNN - FASHION MNIST

In [ ]:
print("\n" + "="*70)
print("ENTRENAMIENTO CNN - FASHION MNIST")
print("="*70)

# Cargar Fashion MNIST
from tensorflow.keras import datasets

(X_train_fm, y_train_fm), (X_test_fm, y_test_fm) = datasets.fashion_mnist.load_data()

# Normalizar
X_train_fm = X_train_fm.astype('float32') / 255.0
X_test_fm = X_test_fm.astype('float32') / 255.0

# Reshape: agregar canal
X_train_fm = X_train_fm.reshape(-1, 28, 28, 1)
X_test_fm = X_test_fm.reshape(-1, 28, 28, 1)

# Split train/val
idx_split = int(0.85 * len(X_train_fm))
X_val_fm = X_train_fm[idx_split:]
y_val_fm = y_train_fm[idx_split:]
X_train_fm = X_train_fm[:idx_split]
y_train_fm = y_train_fm[:idx_split]

print(f"\nDatos Fashion MNIST:")
print(f"  Train: {X_train_fm.shape}")
print(f"  Val:   {X_val_fm.shape}")
print(f"  Test:  {X_test_fm.shape}")

# Construir y entrenar modelo CNN
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model_fm = build_cnn_model((28, 28, 1), 10)

device_info = get_device_info()
print(f"\n🎯 Entrenando en: {device_info['device']}")

start = time.time()
history_fm = model_fm.fit(X_train_fm, y_train_fm, validation_data=(X_val_fm, y_val_fm), epochs=10, batch_size=128, verbose=1)
time_fm = time.time() - start
loss_fm, acc_fm = model_fm.evaluate(X_test_fm, y_test_fm, verbose=0)

print(f"\nAccuracy: {acc_fm:.4f} | Loss: {loss_fm:.4f} | Time: {time_fm:.2f}s")
print("✅ Fashion MNIST training completed")

---
## 5️⃣ ENTRENAR CNN - CIFAR-10

In [ ]:
print("\n" + "="*70)
print("ENTRENAMIENTO CNN - CIFAR-10")
print("="*70)

# Cargar CIFAR-10
(X_train_c10, y_train_c10), (X_test_c10, y_test_c10) = datasets.cifar10.load_data()
y_train_c10 = y_train_c10.flatten()
y_test_c10 = y_test_c10.flatten()

# Normalizar
X_train_c10 = X_train_c10.astype('float32') / 255.0
X_test_c10 = X_test_c10.astype('float32') / 255.0

# Split train/val
idx_split = int(0.85 * len(X_train_c10))
X_val_c10 = X_train_c10[idx_split:]
y_val_c10 = y_train_c10[idx_split:]
X_train_c10 = X_train_c10[:idx_split]
y_train_c10 = y_train_c10[:idx_split]

print(f"\nDatos CIFAR-10:")
print(f"  Train: {X_train_c10.shape}")
print(f"  Val:   {X_val_c10.shape}")
print(f"  Test:  {X_test_c10.shape}")

# Construir y entrenar modelo
model_c10 = build_cnn_model((32, 32, 3), 10)

print(f"\n🎯 Entrenando en: {device_info['device']}")

start = time.time()
history_c10 = model_c10.fit(X_train_c10, y_train_c10, validation_data=(X_val_c10, y_val_c10), epochs=10, batch_size=128, verbose=1)
time_c10 = time.time() - start
loss_c10, acc_c10 = model_c10.evaluate(X_test_c10, y_test_c10, verbose=0)

print(f"\nAccuracy: {acc_c10:.4f} | Loss: {loss_c10:.4f} | Time: {time_c10:.2f}s")
print("✅ CIFAR-10 training completed")

---
## 6️⃣ ENTRENAR LSTM - ECG5000

In [ ]:
print("\n" + "="*70)
print("ENTRENAMIENTO ECG5000")
print("="*70)

# Generar datos sintéticos ECG (similar a ECG5000)
np.random.seed(42)
n_samples_ecg = 5000
timesteps = 96
features = 1
num_classes_ecg = 5

# Generar datos sintéticos
X_ecg = np.random.randn(n_samples_ecg, timesteps, features).astype(np.float32)
y_ecg = np.random.randint(0, num_classes_ecg, n_samples_ecg)

# Normalizar
scaler = StandardScaler()
X_ecg_reshaped = X_ecg.reshape(-1, features)
X_ecg_reshaped = scaler.fit_transform(X_ecg_reshaped)
X_ecg = X_ecg_reshaped.reshape(n_samples_ecg, timesteps, features)

# Split: 70% train, 15% val, 15% test
idx_train = int(0.7 * n_samples_ecg)
idx_val = int(0.85 * n_samples_ecg)

X_train_ecg = X_ecg[:idx_train]
y_train_ecg = y_ecg[:idx_train]
X_val_ecg = X_ecg[idx_train:idx_val]
y_val_ecg = y_ecg[idx_train:idx_val]
X_test_ecg = X_ecg[idx_val:]
y_test_ecg = y_ecg[idx_val:]

print(f"\nDatos ECG:")
print(f"  Train: {X_train_ecg.shape}")
print(f"  Val:   {X_val_ecg.shape}")
print(f"  Test:  {X_test_ecg.shape}")

# Construir y entrenar modelo
model_ecg = build_lstm_model(
    input_shape=(timesteps, features),
    num_classes=num_classes_ecg,
    lstm_units=64
)

device_info = get_device_info()
print(f"\n🎯 Entrenando en: {device_info['device']}")

history_ecg, acc_ecg, loss_ecg, time_ecg = train_and_evaluate_lstm(
    model_ecg, X_train_ecg, y_train_ecg, X_val_ecg, y_val_ecg, X_test_ecg, y_test_ecg,
    epochs=50
)

print("\n✅ ECG5000 training completed")

---
## 7️⃣ ENTRENAR LSTM - UCI HAR

In [ ]:
print("\n" + "="*70)
print("ENTRENAMIENTO UCI HAR")
print("="*70)

# Descargar UCI HAR dataset
print("\nDescargando UCI HAR dataset...")
url_har = "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip"
zip_path = "/tmp/har.zip"
extract_path = "/tmp/UCI_HAR"

try:
    urllib.request.urlretrieve(url_har, zip_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ Dataset descargado")
except Exception as e:
    print(f"⚠️ Error en descarga: {e}")
    print("Usando datos sintéticos para UCI HAR...")

# Cargar datos
def load_har_data():
    try:
        X_train = np.loadtxt(f"{extract_path}/UCI HAR Dataset/train/X_train.txt")
        y_train = np.loadtxt(f"{extract_path}/UCI HAR Dataset/train/y_train.txt", dtype=int) - 1
        X_test = np.loadtxt(f"{extract_path}/UCI HAR Dataset/test/X_test.txt")
        y_test = np.loadtxt(f"{extract_path}/UCI HAR Dataset/test/y_test.txt", dtype=int) - 1
        return X_train, y_train, X_test, y_test
    except:
        print("Generando datos sintéticos para HAR...")
        n_train = 7352
        n_test = 2947
        n_features = 561
        n_classes = 6
        X_train = np.random.randn(n_train, n_features).astype(np.float32)
        y_train = np.random.randint(0, n_classes, n_train)
        X_test = np.random.randn(n_test, n_features).astype(np.float32)
        y_test = np.random.randint(0, n_classes, n_test)
        return X_train, y_train, X_test, y_test

X_train_har, y_train_har, X_test_har, y_test_har = load_har_data()

# Normalizar
scaler_har = StandardScaler()
X_train_har = scaler_har.fit_transform(X_train_har).astype(np.float32)
X_test_har = scaler_har.transform(X_test_har).astype(np.float32)

# Reshape para LSTM (128 timesteps, features reducidas)
timesteps_har = 128
X_train_har_lstm = X_train_har[:, :timesteps_har].reshape(-1, timesteps_har, 1)
X_test_har_lstm = X_test_har[:, :timesteps_har].reshape(-1, timesteps_har, 1)

# Split train/val
idx_split = int(0.85 * len(X_train_har_lstm))
X_val_har = X_train_har_lstm[idx_split:]
y_val_har = y_train_har[idx_split:]
X_train_har = X_train_har_lstm[:idx_split]
y_train_har = y_train_har[:idx_split]

print(f"\nDatos UCI HAR:")
print(f"  Train: {X_train_har.shape}")
print(f"  Val:   {X_val_har.shape}")
print(f"  Test:  {X_test_har_lstm.shape}")

# Construir y entrenar modelo
num_classes_har = 6
model_har = build_lstm_model(
    input_shape=(timesteps_har, 1),
    num_classes=num_classes_har,
    lstm_units=64
)

print(f"\n🎯 Entrenando en: {device_info['device']}")

history_har, acc_har, loss_har, time_har = train_and_evaluate_lstm(
    model_har, X_train_har, y_train_har, X_val_har, y_val_har, X_test_har_lstm, y_test_har,
    epochs=30
)

print("\n✅ UCI HAR training completed")

---
## 8️⃣ COMPARAR RESULTADOS

In [ ]:
# Resumen de resultados
print("\n" + "="*70)
print("RESUMEN DE RESULTADOS")
print("="*70)

print("\nFASE 1 - CNN:")
print(f"  Fashion MNIST - Accuracy: {acc_fm:.4f}, Loss: {loss_fm:.4f}, Time: {time_fm:.2f}s")
print(f"  CIFAR-10      - Accuracy: {acc_c10:.4f}, Loss: {loss_c10:.4f}, Time: {time_c10:.2f}s")

print("\nFASE 2 - LSTM:")
print(f"  ECG5000 - Accuracy: {acc_ecg:.4f}, Loss: {loss_ecg:.4f}, Time: {time_ecg:.2f}s")
print(f"  UCI HAR - Accuracy: {acc_har:.4f}, Loss: {loss_har:.4f}, Time: {time_har:.2f}s")

print(f"\nTiempo total: {(time_fm + time_c10 + time_ecg + time_har):.2f}s")
print(f"Dispositivo: {device_info['device']}")

---
## 9️⃣ GENERAR GRÁFICAS

In [ ]:
# Crear gráficas de training
fig, axes = plt.subplots(2, 4, figsize=(18, 10))
fig.suptitle('TFM - CNN + LSTM GPU Training Results', fontsize=16, fontweight='bold')

# Row 1: CNN
# Fashion MNIST Accuracy
axes[0, 0].plot(history_fm.history['accuracy'], label='Train', linewidth=2)
axes[0, 0].plot(history_fm.history['val_accuracy'], label='Val', linewidth=2)
axes[0, 0].set_title('Fashion MNIST - Accuracy', fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Fashion MNIST Loss
axes[0, 1].plot(history_fm.history['loss'], label='Train', linewidth=2)
axes[0, 1].plot(history_fm.history['val_loss'], label='Val', linewidth=2)
axes[0, 1].set_title('Fashion MNIST - Loss', fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# CIFAR-10 Accuracy
axes[0, 2].plot(history_c10.history['accuracy'], label='Train', linewidth=2)
axes[0, 2].plot(history_c10.history['val_accuracy'], label='Val', linewidth=2)
axes[0, 2].set_title('CIFAR-10 - Accuracy', fontweight='bold')
axes[0, 2].set_xlabel('Epoch')
axes[0, 2].set_ylabel('Accuracy')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# CIFAR-10 Loss
axes[0, 3].plot(history_c10.history['loss'], label='Train', linewidth=2)
axes[0, 3].plot(history_c10.history['val_loss'], label='Val', linewidth=2)
axes[0, 3].set_title('CIFAR-10 - Loss', fontweight='bold')
axes[0, 3].set_xlabel('Epoch')
axes[0, 3].set_ylabel('Loss')
axes[0, 3].legend()
axes[0, 3].grid(True, alpha=0.3)

# Row 2: LSTM
# ECG5000 Accuracy
axes[1, 0].plot(history_ecg.history['accuracy'], label='Train', linewidth=2)
axes[1, 0].plot(history_ecg.history['val_accuracy'], label='Val', linewidth=2)
axes[1, 0].set_title('ECG5000 - Accuracy', fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# ECG5000 Loss
axes[1, 1].plot(history_ecg.history['loss'], label='Train', linewidth=2)
axes[1, 1].plot(history_ecg.history['val_loss'], label='Val', linewidth=2)
axes[1, 1].set_title('ECG5000 - Loss', fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# UCI HAR Accuracy
axes[1, 2].plot(history_har.history['accuracy'], label='Train', linewidth=2)
axes[1, 2].plot(history_har.history['val_accuracy'], label='Val', linewidth=2)
axes[1, 2].set_title('UCI HAR - Accuracy', fontweight='bold')
axes[1, 2].set_xlabel('Epoch')
axes[1, 2].set_ylabel('Accuracy')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

# UCI HAR Loss
axes[1, 3].plot(history_har.history['loss'], label='Train', linewidth=2)
axes[1, 3].plot(history_har.history['val_loss'], label='Val', linewidth=2)
axes[1, 3].set_title('UCI HAR - Loss', fontweight='bold')
axes[1, 3].set_xlabel('Epoch')
axes[1, 3].set_ylabel('Loss')
axes[1, 3].legend()
axes[1, 3].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Gráficas guardadas: training_results.png")

---
## 🔟 GUARDAR Y DESCARGAR RESULTADOS

In [ ]:
# Crear resumen CSV
results_df = pd.DataFrame({
    'Phase': ['Fase 1 (CNN)', 'Fase 1 (CNN)', 'Fase 2 (LSTM)', 'Fase 2 (LSTM)'],
    'Dataset': ['Fashion MNIST', 'CIFAR-10', 'ECG5000', 'UCI HAR'],
    'Model': ['CNN', 'CNN', 'LSTM', 'LSTM'],
    'Accuracy': [acc_fm, acc_c10, acc_ecg, acc_har],
    'Loss': [loss_fm, loss_c10, loss_ecg, loss_har],
    'Training_Time_s': [time_fm, time_c10, time_ecg, time_har],
    'Device': ['GPU', 'GPU', 'GPU', 'GPU']
})

results_df.to_csv('gpu_results.csv', index=False)
print("✅ Resultados guardados: gpu_results.csv")
print("\n" + results_df.to_string())

---

## 📥 DESCARGAR ARCHIVOS

In [ ]:
# Descargar archivos desde Colab
try:
    from google.colab import files
    
    print("\n" + "="*70)
    print("DESCARGANDO RESULTADOS")
    print("="*70)
    
    files_to_download = ['gpu_results.csv', 'training_results.png']
    
    for file in files_to_download:
        try:
            files.download(file)
            print(f"✅ Descargado: {file}")
        except Exception as e:
            print(f"⚠️ No se pudo descargar {file}: {e}")
    
    print("\n" + "="*70)
    print("✅ ENTRENAMIENTO COMPLETADO")
    print("="*70)
    print("\nResultados guardados en:")
    print("  📊 gpu_results.csv")
    print("  📈 training_results.png")
except:
    print("\n⚠️ No estás en Colab. Los archivos se han creado localmente.")
    print("   - gpu_results.csv")
    print("   - training_results.png")